In [ ]:
import geopandas as gpd
import os
import matplotlib.pyplot as plt
import boto3
import sys
import xarray as xr
import pandas as pd
from functools import wraps
import dask_geopandas
import re
sys.path.append(os.path.expanduser('../../'))
from scripts.utils.write_metadata import (
    append_metadata
)
import warnings
warnings.filterwarnings("ignore")

In [ ]:
shp_fname = 'zip+s3://ca-climate-index/1_pull_data/climate_risk/flood/exposure/koordinates/california-fema-100-year-floodplains.gdb.zip'
# varname = fname_dict[fname]
df = gpd.read_file(shp_fname)

In [ ]:
# read in CA census tiger file
census_shp_dir = "s3://ca-climate-index/0_map_data/2021_tiger_census_tract/2021_ca_tract/"
ca_boundaries = gpd.read_file(census_shp_dir)
# need to rename columns so we don't have any duplicates in the final geodatabase
column_names = ca_boundaries.columns
new_column_names = ["USCB_"+column for column in column_names if column != "geometry"]
ca_boundaries = ca_boundaries.rename(columns=dict(zip(column_names, new_column_names)))
ca_boundaries = ca_boundaries.to_crs(3310)
df = df.to_crs(ca_boundaries.crs)

In [ ]:
df = df.dropna()
ddf = df[df.FloodZone.str.contains("100")]  
ddf

In [ ]:
ca_boundaries = ca_boundaries.to_crs(3310)
ddf = ddf.to_crs(ca_boundaries.crs)
ddf = ddf[["geometry","FLD_ZONE","FloodZone"]]

In [ ]:
over_df = gpd.overlay(ca_boundaries, ddf, how="intersection")

In [ ]:
gdf_new = over_df.dissolve(by='USCB_GEOID')
gdf_new

In [ ]:
floodzone_gdf = gdf_new[["geometry"]]
floodzone_gdf["floodzone_area"] = floodzone_gdf.area
floodzone_gdf

In [ ]:
tract_gdf = ca_boundaries[["USCB_GEOID","geometry"]]
tract_gdf["tract_area"] = tract_gdf.area
tract_gdf

In [ ]:
merged_df = pd.merge(tract_gdf, floodzone_gdf, on="USCB_GEOID", suffixes=["_tract","_floodzone"])
merged_df["%_floodzone"] = (merged_df["floodzone_area"]/merged_df["tract_area"]) * 100

In [ ]:
to_plot = merged_df[["geometry_tract","%_floodzone","USCB_GEOID"]]
pl_gdf = gpd.GeoDataFrame(to_plot, geometry="geometry_tract")
pl_gdf.explore(column="%_floodzone", legend=True)